In [1]:
from PyPDF2 import PdfFileReader, PdfFileWriter
import re
import json

In [2]:
with open("pdf/denkmalliste_berlin_092020.txt", "r", encoding='UTF-8') as file:
    txt = file.read()
    txt = txt.replace("\n", " ")
    txt = ' '.join(txt.split())
    txt = txt.replace(") + (", ", ")

In [18]:
R1=r'[(]\w+[-]\w+[/]\w+[-][A-Z][)]|[(]\w+[/]\w+[-][A-Z][)]'
R1=R1+r'|[(]\w+[-]\w+[/]\w+[-][A-Z][,]\s\w+[-]\w+[/]\w+[-][A-Z][)]'
R1=R1+r'|[(]\w+[/]\w+[-][A-Z][,]\s\w+[/]\w+[-][A-Z][)]'
R1=R1+r'|[(]\w+[-]\w+[/]\w+[-][A-Z][a-z][)]|[(]\w+[/]\w+[-][A-Z][a-z][)]'
R2='\(.*?\)'
T1=r'Denkmalbereiche\s[(]Gesamtanlagen[)]'
T2=r'Denkmalbereiche\s[(]Ensembles[)]'
T3=r'Baudenkmale'
T4=r'Gartendenkmale'
T5=r'Bodendenkmale'
T6=r'Bodendenkmal'
bezirke=['Charlottenburg[-]Wilmersdorf', 'Friedrichshain[-]Kreuzberg', 'Lichtenberg', 'Marzahn[-]Hellersdorf', 'Mitte', 'Neukölln', 'Pankow', 'Reinickendorf', 'Spandau', 'Steglitz[-]Zehlendorf', 'Tempelhof[-]Schöneberg', 'Treptow[-]Köpenick']
t0='Konstituierende\sBestandteile\sdes\sEnsembles:'
t1='Nicht\skonstituierender\sBestandteil\sdes\sEnsembles:'
t2='Nicht\skonstituierende\sBestandteile\sdes\sEnsembles:'
t3='Weiterer\sBestandteil\sdes\sEnsembles:'
t4='Weitere\sBestandteile\sdes\sEnsembles:'
t5='Bestandteile\sdes\sEnsembles:'
t6='Ensembles\ssiehe'
t7='Gesamtanlage\ssiehe:'
t8='Gesamtanlagen\ssiehe:'
t9='Baudenkmal\ssiehe:'
t10='Baudenkmale\ssiehe:'
t11='Gartendenkmal\ssiehe:'
t12='Gartendenkmale\ssiehe:'
t13='Bodenkmal\ssiehe:'
t14='Bodenkmale\ssiehe:'

In [19]:
#Helper functions 
def findAllObjNr(s):
    pat = r'(\w*%s\w*)' % '\d{8}'
    list=  re.findall(pat, s)
    return list

def findMainObjNr(s):
    pat = r'('+R1+'|'+T1+'|'+T2+'|'+T3+'|'+T4+'|'+T5+'|'+T6+')'
    #list= re.findall(pat+r'(\s\d+)', s)# return as a whole string
    list = re.findall(r'%s(\s\d+)' % pat, s)# seperates the number from pat
    mainObjNr=[]
    for item in list:
        mainObjNr.append(item[1].replace(' ',''))
        #mainObjNr.append(item)
    #print('mainObjNr: ', mainObjNr)
    return mainObjNr

def findMainObjNrBezirk(s):
    mainObjNr=findMainObjNr(s)
    pat = r'('+R1+')'
    mainBezirk= re.findall(pat, s)
    mainObjNrBezirk={}
    for i in range(len(mainObjNr)):    
        k=i
        if 4532<=i:# for 09011691
            k=i+1
        if 6885<=i:# for 09046008
            k=i+2
        if 8189<=i:# for 09010223
            k=k+1
        mainObjNrBezirk[mainObjNr[i]]=mainBezirk[k]
    print('len(mainBezirk): ',len(mainBezirk)-3,' len(mainObjNr): ', len(mainObjNr))
    return mainObjNrBezirk

def findDenkmalKategorie(s):
    pat = r'('+T1+'|'+T2+'|'+T3+'|'+T4+'|'+T5+'|'+T6+')'
    list2 = re.findall(r'%s(\s\d+)' % pat, s)# seperates the number from pat
    denkmalKategorie={}
    for item in list2:
        denkmalKategorie.setdefault(item[0].replace('Bodendenkmal ','Bodendenkmale '),[]).append(item[1].replace(' ',''))
    #print('denkmalKategorie: ', denkmalKategorie)
    return denkmalKategorie

def get_denkmalKategorie(val, my_dict):
    for key, value in my_dict.items():
         if val in value:
             return [key,True]
    return ["keine Kategorie gefunden",False]

def removeR2(s):
    for el in re.findall(R2, s):
        if 'siehe' in el:
            s=s.replace(el,"")
    return s

def findSubObjNr(s):# subObjNr are all ObjNr related to a "Ensemble"
    mainObjNr=findMainObjNr(s)
    allObjNr=findAllObjNr(s)
    subObjNr=[]
    for item in allObjNr:
        if item not in mainObjNr:
            subObjNr.append(item)
    print(findMainObjNr(s))
    print('strBezirke:', strBezirke)
    print(mainObjNr)
    print(allObjNr)
    print(subObjNr)
    return 0

In [20]:
x='09011553 Deidesheimer Straße 20-24, (siehe Denkmalliste Neukölln, Ensemble Platz der Luftbrücke 1-6) Wohnanlage 1925-26 von (siehe Ensemble Franklinstraße 11-15A) Heinrich Straumer und Richard Binder (siehe Ensemble Rüdesheimer Platz 1-5..., Teilbereich II...) Ahrweilerstraße 34-36 Homburger Straße 6, 10Laubacher Straße 38(CHA-WIL/WILMD-G)'
print(removeR2(x))

09011553 Deidesheimer Straße 20-24,  Wohnanlage 1925-26 von  Heinrich Straumer und Richard Binder  Ahrweilerstraße 34-36 Homburger Straße 6, 10Laubacher Straße 38(CHA-WIL/WILMD-G)


In [13]:
#here the main process is performed to extract required information
def preprocess(linkMainObjNr,value,end,denkmalKategorie,mainObjNrBezirk,linkMainObjNr2,dictObjNrValue):
    pat=r'('+t0+'|'+t1+'|'+t2+'|'+t3+'|'+t4+'|'+t5+'|'+t6+'|'+t7+'|'+t8+'|'+t9+'|'+t10+'|'+t11+'|'+t12+'|'+t13+'|'+t14+')'# search pattern
    searcher = re.search(pat, value)
    if bool(searcher):
        n=0
        while bool(searcher):# continue extracting for searching the pattern above
            #print('allObjNr[i]: ', allObjNr[i], ' value: ', value)
            #print('n: ', n)
            startInhalt= 0
            endInhalt = searcher.span()[0]
            if n==0:
                valueInhalt = value[startInhalt:endInhalt]
                #R2 contains "bezirke" which is disadvantageous for searching an adress
                valueInhalt=removeR2(valueInhalt)
                dictObjNrValue[linkMainObjNr]['Inhalt']=valueInhalt
            else:
                value2 = value[startInhalt:endInhalt]
                #R2 contains "bezirke" which is disadvantageous for searching an adress
                value2 = removeR2(value2)
                dictObjNrValue[linkMainObjNr][valueKey]=value2
            #######
            startKey = searcher.span()[0]
            endKey = searcher.span()[1]
            valueKey = value[startKey:endKey]
            ################################Unification of keys
            valueKey = valueKey.replace("Gesamtanlage ", "Gesamtanlagen ")
            valueKey = valueKey.replace("Baudenkmal ", "Baudenkmale ")
            valueKey = valueKey.replace("Bodenkmal ", "Bodenkmale ")
            valueKey = valueKey.replace("Gartendenkmal ", "Gartendenkmale ")
            valueKey = valueKey.replace("Weiterer Bestandteil des Ensembles:", "Bestandteile des Ensembles:")
            valueKey = valueKey.replace("Weitere Bestandteile des Ensembles:", "Bestandteile des Ensembles:")
            valueKey = valueKey.replace("Bestandteile des Ensembles:", "Bestandteile des Ensembles:")
            valueKey = valueKey.replace("Konstituierende Bestandteile des Ensembles:", "Bestandteile des Ensembles:")
            valueKey = valueKey.replace("Nicht konstituierender Bestandteil des Ensembles:", "Nicht konstituierende Bestandteile des Ensembles:")
            valueKey = valueKey.replace(":", "")
            valueKey = valueKey.replace(" siehe", "") 
            ################################End unification of keys
            value = value[endKey:end]
            searcher = re.search(pat, value)
            n+=1
        else:
            if valueKey=='Nicht konstituierende Bestandteile des Ensembles':
                value2 = value.replace(mainObjNrBezirk[linkMainObjNr2], '')
                #R2 contains "bezirke" which is disadvantageous for searching an adress
                value2=removeR2(value2)
                dictObjNrValue[linkMainObjNr][valueKey]=value2
    else:# if there is no search pattern condition is fullfilled. Mostly given for "Bodendenkmale"
        value2=value.replace(mainObjNrBezirk[linkMainObjNr2], '')
        #R2 contains "bezirke" which is disadvantageous for searching an adress
        value2=removeR2(value2)
        dictObjNrValue[linkMainObjNr]['Inhalt']= value2
    return 0

In [14]:
#sort and extracts the desired data and creates a dictionary
def findObjNrValue(s):# should return a dictionary with ObjectNr as keys and the associated value will be a new dictionary with additinaly atttributes such as "Bezirk", "Ensemble","Gartendenkmal" ...
    allObjNr=findAllObjNr(s)
    mainObjNrBezirk=findMainObjNrBezirk(s)
    mainObjNr=findMainObjNr(s)#mainObjNrBezirk.keys()
    denkmalKategorie=findDenkmalKategorie(s)
    dictObjNrValue={}
    #iterate over all ObjNr
    for i in range(len(allObjNr)):
        dictObjNrValue[allObjNr[i]]={}
        start = s.find(allObjNr[i]) + len(allObjNr[i])
        #this condition is important for the last element since another ObjNr will not follow
        if i !=len(allObjNr)-1:
            end = s.find(allObjNr[i+1])
        #for the last element there is no further upcoming objnr
        else: 
            end = len(s)
        value = s[start:end]
        ####################################CONTINUE HERE!!!
        ##dont forget special case for 09011506 Rüdesheuer Platz  -> Teilbereich 1-3
        if (allObjNr[i] in mainObjNr):# here consider only the mainObjNr
            linkMainObjNr=allObjNr[i]# to remember the last mainObjNr
            dictObjNrValue[linkMainObjNr]['main']=True
            dictObjNrValue[linkMainObjNr]['sub']=False
            dictObjNrValue[linkMainObjNr]['linkSubObjNr']=[]# for the aissgnment to sub ObjNr since a mainObjNr can have multiple subObjNr
            dictObjNrValue[linkMainObjNr]['Bestandteile des Ensembles']=[]# for the aissgnment to sub ObjNr since a mainObjNr can have multiple subObjNr
            dictObjNrValue[linkMainObjNr]['Bezirk']=mainObjNrBezirk[linkMainObjNr]
            dumy = preprocess(linkMainObjNr,value,end,denkmalKategorie,mainObjNrBezirk,linkMainObjNr,dictObjNrValue)
            ##assign a denkmal category
            getDenkmal=get_denkmalKategorie(linkMainObjNr, denkmalKategorie)# return a list. first element is the categorie if there is a match, second return a true if match is present.
            if getDenkmal[1]:
                assignDenkmal=getDenkmal[0]
        else:
            dictObjNrValue[linkMainObjNr]['linkSubObjNr'].append(allObjNr[i])
            dictObjNrValue[linkMainObjNr]['Bestandteile des Ensembles'].append(allObjNr[i])
            #here consider the subObjNr
            dictObjNrValue[allObjNr[i]]['main']=False
            dictObjNrValue[allObjNr[i]]['sub']=True
            dictObjNrValue[allObjNr[i]]['linkMainObjNr']=linkMainObjNr
            dictObjNrValue[allObjNr[i]]['Bezirk']= mainObjNrBezirk[linkMainObjNr]
            #consider also "Nicht konstituierende Bestandteile des Ensembles" for the subObjNr
            dumy = preprocess(allObjNr[i],value,end,denkmalKategorie,mainObjNrBezirk,linkMainObjNr,dictObjNrValue)
            #dictObjNrValue[allObjNr[i]]['Inhalt']= value.replace(mainObjNrBezirk[linkMainObjNr], '')
            # these are appended to the mainObjNr dictionary
        dictObjNrValue[allObjNr[i]]['Denkmalart']=assignDenkmal
        #########################Test
        mainBezirk= re.findall(r'('+R1+')', value)
        if (len(mainBezirk)!=0) and mainObjNrBezirk[linkMainObjNr] not in value:
            dictObjNrValue[allObjNr[i]]['mistake']='here'
        ##########################
        #print('\n','allObjNr[i]: ',allObjNr[i],' dictObjNrValue: ', dictObjNrValue[allObjNr[i]],'\n')
        #print('dictObjNrValue: ', dictObjNrValue[allObjNr[i]])
    return dictObjNrValue

In [15]:
#save objNrValue as json file
with open('ObjNrValue4.json', 'w') as fp:
    json.dump(findObjNrValue(txt), fp,  indent=4,ensure_ascii=False)#.encode('utf8')
print('successfully saved')

len(mainBezirk):  8216  len(mainObjNr):  8216
successfully saved
